In [ ]:
import pennylane as qml
import pennylane.numpy as np

In [ ]:

def calculate_pa(c, a_arr):
  out = []
  for a in a_arr:
    Ua = np.array([[a, np.sqrt(1-a**2)], [np.sqrt(1-a**2), -a]])

    S = np.array([[np.exp(c[0]*1j), 0], [0, np.exp(-c[0]*1j)]])
    for i in range(1, len(c)):
      tmp_s = np.array([[np.exp(c[i]*1j), 0], [0, np.exp(-c[i]*1j)]])
      tmp = np.matmul(Ua, tmp_s)
      S = np.matmul(S, tmp)

    out.append(S[0][0])

  return out

def cost_f(a, params, y):
  out = (a**3)*params[0] + a*params[1]
  return (out - y)**2


def coefficients(angles):

# prepare data for training
    a_arr = np.arange(-1, 1+0.1, 0.1, requires_grad=False)
    pa = calculate_pa(angles, a_arr)
    pa_real = []
    pa_imag = []

    for elem in pa:
      pa_real.append(elem.real)
      pa_imag.append(elem.imag)


    pa_real = np.array(pa_real, requires_grad=False)
    pa_imag = np.array(pa_imag, requires_grad=False)

# train real part
    opt = qml.GradientDescentOptimizer(stepsize=0.1)
    real_part = np.array([0.0, 0.0], requires_grad=True)

    epochs = 200

    for epoch in range(epochs):
      for i in range(len(a_arr)):
        real_part = opt.step(cost_f, a_arr[i], real_part, pa_real[i])[1]

# train imaginary part
    opt = qml.GradientDescentOptimizer(stepsize=0.1)
    imag_part = np.array([0.0, 0.0], requires_grad=True)

    epochs = 200

    for epoch in range(epochs):
      for i in range(len(a_arr)):
        imag_part = opt.step(cost_f, a_arr[i], imag_part, pa_imag[i])[1]

    alpha_r = float(real_part[0])
    beta_r = float(real_part[1])
    alpha_im = float(imag_part[0])
    beta_im = float(imag_part[1])

    return [alpha_r + alpha_im*1j, beta_r + beta_im*1j]

In [ ]:
angles = [0.5, 0.8, 1.0, 1.0]
coefficients(angles)